In [1]:
import pandas as pd
from natasha import MorphVocab
import numpy as np
import pymorphy2
from slovnet import NER
from navec import Navec


In [2]:
data_initial = pd.read_csv('test_data_local.csv') 
data_initial.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [3]:
data = data_initial.drop(np.where(data_initial['role'] == 'client')[0])
data.head()

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет


In [4]:
morph_vocab = MorphVocab()

In [5]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc,
    PER,
    NamesExtractor,
)


 # Извлечение приветствия и прощания

In [6]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [7]:
def get_lemma(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    res = []    
    for _ in doc.tokens:
        res.append(_.lemma) 
    return res
    

In [8]:
data['lemma'] = data['text'].apply(get_lemma)

In [9]:
data[data['dlg_id'] == 0]

,dlg_id,line_n,role,text,lemma
1,0,1,manager,Алло здравствуйте,"[алло, здравствовать]"
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне..."
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"[угу, ну, возможно, вы, рассмотреть, и, другой..."
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"[угу, а, на, что, вы, обращать, внимание, при,..."
11,0,11,manager,Что для вас приоритет,"[что, для, вы, приоритет]"
15,0,15,manager,Ну у вас срок заканчивается поэтому мы набрали...,"[ну, у, вы, срок, заканчиваться, поэтому, мы, ..."
29,0,29,manager,А так нет не только поэтому просто я обратила ...,"[а, так, нет, не, только, поэтому, просто, я, ..."
34,0,34,manager,А если вы 19 являетесь то лучше то идти бесплатно,"[а, если, вы, 19, являться, то, лучше, тот, ид..."
36,0,36,manager,Ага хорошо,"[ага, хороший]"
45,0,45,manager,Индивидуальным поэтому не все то есть сотрудни...,"[индивидуальный, поэтому, не, весь, тот, быть,..."


In [10]:

greetings = ['здравствуйте','добрый день','добрый вечер','доброе утро', 'привет','Приветствую', 'здравствовать']
bye = ['до свидания','до встречи','всего доброго','всего наилучшего','пока','прощайте','всего хорошего']

def cleaning_pattern(pattern):
    pattern_data = []
    for item in pattern:
        pattern_data.append(list(get_lemma(item)))
    return pattern_data

greetings=cleaning_pattern(greetings)
print(greetings)

bye=cleaning_pattern(bye)
print(bye)


[['здравствуйте'], ['добрый', 'день'], ['добрый', 'вечер'], ['добрый', 'утро'], ['привет'], ['приветствовать'], ['здравствовать']]
[['до', 'свидание'], ['до', 'встреча'], ['весь', 'добрый'], ['весь', 'хороший'], ['пока'], ['прощать'], ['весь', 'хороший']]


In [11]:
def classification(row):
    for item in greetings:
        if len(item) == 1:
            if item[0] in row:
                return 'greeting'
        else:
            if item[0] in row and item[1] in row:
                return 'greeting'
    for item in bye:
        if len(item) == 1:
            if item[0] in row:
                return 'bye'
        else:
            if item[0] in row and item[1] in row:
                return 'bye'        
    

In [12]:
data['classification'] = data['lemma'].apply(classification)

## check 

In [13]:
data.head()

,dlg_id,line_n,role,text,lemma,classification
1,0,1,manager,Алло здравствуйте,"[алло, здравствовать]",greeting
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",None
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"[угу, ну, возможно, вы, рассмотреть, и, другой...",None
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"[угу, а, на, что, вы, обращать, внимание, при,...",None
11,0,11,manager,Что для вас приоритет,"[что, для, вы, приоритет]",None


In [14]:
# data.query('dlg_id == 0 & line_n == 1')
# data.query('dlg_id == 1 & line_n == 1')
# data.query('dlg_id == 1 & line_n == 53')
# data.query('dlg_id == 1 & line_n == 54')
# data.query('dlg_id == 3 & line_n == 1')
# data.query('dlg_id == 3 & line_n == 51')
# data.query('dlg_id == 4 & line_n == 33')
# data.query('dlg_id == 5 & line_n == 142')

# проверка условия и поздоровался и попрощался 

## количество  диалогов 

In [15]:
len(data['dlg_id'].unique())

6

In [16]:
cond = {i:False for i in range(6)}
cond

{0: False, 1: False, 2: False, 3: False, 4: False, 5: False}

In [17]:
for i in range(6):
    cond[i] = (not (data[data['dlg_id'] == i][data['classification'] == 'greeting']).empty) and (not (data[data['dlg_id'] == i][data['classification'] == 'bye']).empty)

cond




/var/folders/0p/znb2qr110hb4zpt4wlkw3w580000gn/T/ipykernel_50040/750795215.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cond[i] = (not (data[data['dlg_id'] == i][data['classification'] == 'greeting']).empty) and (not (data[data['dlg_id'] == i][data['classification'] == 'bye']).empty)


{0: True, 1: True, 2: True, 3: True, 4: False, 5: False}

In [18]:
for k,v in cond.items():
    if v:
        print(f"Менеджер выполнил требование и поздоровался и попрощался с клиентом в диалоге номер {k}")
    else:
        print(f"Менеджер  не выполнил требование в диалоге номер {k}")

Менеджер выполнил требование и поздоровался и попрощался с клиентом в диалоге номер 0
Менеджер выполнил требование и поздоровался и попрощался с клиентом в диалоге номер 1
Менеджер выполнил требование и поздоровался и попрощался с клиентом в диалоге номер 2
Менеджер выполнил требование и поздоровался и попрощался с клиентом в диалоге номер 3
Менеджер  не выполнил требование в диалоге номер 4
Менеджер  не выполнил требование в диалоге номер 5


# извлечение имени

In [19]:
morph = pymorphy2.MorphAnalyzer()

In [20]:
data.head()

,dlg_id,line_n,role,text,lemma,classification
1,0,1,manager,Алло здравствуйте,"[алло, здравствовать]",greeting
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",None
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"[угу, ну, возможно, вы, рассмотреть, и, другой...",None
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"[угу, а, на, что, вы, обращать, внимание, при,...",None
11,0,11,manager,Что для вас приоритет,"[что, для, вы, приоритет]",None


In [21]:
# introduce = ["меня зовут","я + name", "это + name"]
introduce = ["меня зовут","я", "это"]

In [22]:
introduce=cleaning_pattern(introduce)
print(introduce)

[['я', 'звать'], ['я'], ['это']]


In [23]:
def is_name_f(text):
#     print(text)
    is_name = False
    name = False
    introduce_flag = False
    for word in text:
        if any('Name' in p.tag for p in morph.parse(word)):
            is_name = True
            name = word
            for el in introduce:
                if len(el) == 1:
                    if el[0] in text:
                        if text.index(el[0])<len(text) and text[text.index(el[0])+1] == word:
                            introduce_flag=True
                else:
                    if el[0] in text and el[1] in text:
                        introduce_flag=True       
            return (is_name,name,introduce_flag)
    return (is_name,name,introduce_flag)    

In [24]:
def extract_name(row):
    res = is_name_f(row)
    if res[0] and res[-1]:
        print("+")
        return('True/'+res[1])
    else:
        return('False/None')
        
    

In [25]:
data['introduce/name'] = pd.DataFrame(data['lemma'].apply(extract_name))

+
+
+
+
+


## вручную я тоже 5 насчитала реплик с представлением 

In [26]:
data

,dlg_id,line_n,role,text,lemma,classification,introduce/name
1,0,1,manager,Алло здравствуйте,"[алло, здравствовать]",greeting,False/None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",None,True/ангелина
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"[угу, ну, возможно, вы, рассмотреть, и, другой...",None,False/None
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"[угу, а, на, что, вы, обращать, внимание, при,...",None,False/None
11,0,11,manager,Что для вас приоритет,"[что, для, вы, приоритет]",None,False/None
...,...,...,...,...,...,...,...
472,5,135,manager,Так дмитрий,"[так, дмитрий]",None,False/None
473,5,136,manager,Все записала тогда завтра ждите звонка,"[весь, записать, тогда, завтра, ждать, звонок]",None,False/None
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,"[по, повод, виджет, и, с, они, уже, обсудить, ...",None,False/None
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,"[весь, я, вы, высылать, счет, и, с, вы, на, св...",None,False/None


# название компании 

In [27]:
def company_name(row):
    res = None
    if 'компания' in row:
        res = ''
        i = row.index('компания')+1
        while i < len(row):
            if morph.parse(row[i])[0].tag.POS == 'NOUN':
                res = res + row[i] + ' '
                break
            elif morph.parse(row[i])[0].tag.POS == 'ADJF':
                res = res + row[i] + ' '
                while i < len(row):
                    i+=1
                    if morph.parse(row[i])[0].tag.POS == 'NOUN': 
                        res = res + row[i] + ' '
                        return res
                    elif morph.parse(row[i])[0].tag.POS == 'ADJF':
                        res = res + row[i] + ' '
                        i+=1
                    else:
                        return res
            else:
                break
    return res           
        
        

In [28]:
data['company_name'] = data['lemma'].apply(company_name)

In [29]:
data.head()

,dlg_id,line_n,role,text,lemma,classification,introduce/name,company_name
1,0,1,manager,Алло здравствуйте,"[алло, здравствовать]",greeting,False/None,None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",None,True/ангелина,диджитал бизнес
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"[угу, ну, возможно, вы, рассмотреть, и, другой...",None,False/None,None
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"[угу, а, на, что, вы, обращать, внимание, при,...",None,False/None,None
11,0,11,manager,Что для вас приоритет,"[что, для, вы, приоритет]",None,False/None,None


## check

In [30]:
# data.query('dlg_id == 0 & line_n == 3')
# data.query('dlg_id == 1 & line_n == 2')
# data.query('dlg_id == 2 & line_n == 3')
data.query('dlg_id == 3 & line_n == 2')

,dlg_id,line_n,role,text,lemma,classification,introduce/name,company_name
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,"[добрый, я, максим, звать, компания, китобизне...",None,True/максим,китобизнес


In [31]:
data.to_csv('answer.csv')